### =====================================================================
### IMPORTACIÓN GENERAL DE LA INFORMACIÓN.
### =====================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
from urllib2 import urlopen

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [2]:
#lEEMOS EL CSV
train = pd.read_csv('../../DATA_TP2/train.csv')
test = pd.read_csv('../../DATA_TP2/test.csv')
respuesta = pd.read_csv('../../DATA_TP2/ejemploRespuesta.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 22 columns):
id                            60000 non-null int64
titulo                        58622 non-null object
descripcion                   59599 non-null object
tipodepropiedad               59993 non-null object
direccion                     46809 non-null object
ciudad                        59917 non-null object
provincia                     59958 non-null object
antiguedad                    49286 non-null float64
habitaciones                  54372 non-null float64
garages                       50677 non-null float64
banos                         53446 non-null float64
metroscubiertos               55701 non-null float64
metrostotales                 47345 non-null float64
idzona                        52821 non-null float64
lat                           29305 non-null float64
lng                           29305 non-null float64
fecha                         60000 non-null object

### =====================================================================
### PREPARACIÓN DE DATOS, FEATURE ENGINEERING.
### =====================================================================

In [5]:
train['lat'].fillna(0, inplace=True)
train['lng'].fillna(0, inplace=True)

train['titulo'] = train['titulo'].str.upper()
train['descripcion'] = train['descripcion'].str.upper()
train['direccion'] = train['direccion'].str.upper()
train['ciudad'] = train['ciudad'].str.upper()
train['provincia'] = train['provincia'].str.upper()
train['tipodepropiedad'] = train['tipodepropiedad'].str.upper()

train['provincia'] = train['provincia'].str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')

train['ciudad'] = train['ciudad'].str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')

In [6]:
train['excelente'] = 0
train.loc[train.descripcion.str.contains('VIGILANCIA', na=False), 'vigilancia'] = 1
train['vigilancia'] = 0
train.loc[train.descripcion.str.contains('EXCELENTE', na=False), 'excelente'] = 1
train['esquina'] = 0
train.loc[train.descripcion.str.contains('ESQUINA', na=False), 'esquina'] = 1
train['remodelada'] = 0
train.loc[train.descripcion.str.contains('REMODELADA', na=False), 'remodelada'] = 1
train['lujo'] = 0
train.loc[train.descripcion.str.contains('LUJO', na=False), 'lujo'] = 1

In [7]:
# AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
train['antiguedad_agrupada'] = 0

train.loc[train.antiguedad <= 5, 'antiguedad_agrupada'] = 0
train.loc[train.antiguedad > 95, 'antiguedad_agrupada'] = 100
train.loc[((train.antiguedad > 5) & (train.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
train.loc[((train.antiguedad > 15) & (train.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
train.loc[((train.antiguedad > 25) & (train.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
train.loc[((train.antiguedad > 35) & (train.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
train.loc[((train.antiguedad > 45) & (train.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
train.loc[((train.antiguedad > 55) & (train.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
train.loc[((train.antiguedad > 65) & (train.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
train.loc[((train.antiguedad > 75) & (train.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
train.loc[((train.antiguedad > 85) & (train.antiguedad <= 95)), 'antiguedad_agrupada'] = 90

In [8]:
# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
train['metroscubiertos_agrupada'] = 0

train.loc[train.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 0
train.loc[train.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 1000
train.loc[((train.metroscubiertos > 5) & (train.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 10
train.loc[((train.metroscubiertos > 15) & (train.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 20
train.loc[((train.metroscubiertos > 25) & (train.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 30
train.loc[((train.metroscubiertos > 35) & (train.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 40
train.loc[((train.metroscubiertos > 45) & (train.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 50
train.loc[((train.metroscubiertos > 55) & (train.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 60
train.loc[((train.metroscubiertos > 65) & (train.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 70
train.loc[((train.metroscubiertos > 75) & (train.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 80
train.loc[((train.metroscubiertos > 85) & (train.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 90
train.loc[((train.metroscubiertos > 95) & (train.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 100
train.loc[((train.metroscubiertos > 105) & (train.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 110
train.loc[((train.metroscubiertos > 115) & (train.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 125
train.loc[((train.metroscubiertos > 130) & (train.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 150
train.loc[((train.metroscubiertos > 155) & (train.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 200
train.loc[((train.metroscubiertos > 205) & (train.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 250
train.loc[((train.metroscubiertos > 255) & (train.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 300
train.loc[((train.metroscubiertos > 305) & (train.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 350
train.loc[((train.metroscubiertos > 355) & (train.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 400
train.loc[((train.metroscubiertos > 405) & (train.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 450
train.loc[((train.metroscubiertos > 455) & (train.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 500
train.loc[((train.metroscubiertos > 505) & (train.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 550
train.loc[((train.metroscubiertos > 555) & (train.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 600
train.loc[((train.metroscubiertos > 605) & (train.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 650
train.loc[((train.metroscubiertos > 655) & (train.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 700
train.loc[((train.metroscubiertos > 705) & (train.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 750
train.loc[((train.metroscubiertos > 755) & (train.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 800

In [9]:
# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
train['metrostotales_agrupada'] = 0

train.loc[train.metrostotales <= 5, 'metrostotales_agrupada'] = 0
train.loc[train.metrostotales > 805, 'metrostotales_agrupada'] = 1000
train.loc[((train.metrostotales > 5) & (train.metrostotales <= 15)), 'metrostotales_agrupada'] = 10
train.loc[((train.metrostotales > 15) & (train.metrostotales <= 25)), 'metrostotales_agrupada'] = 20
train.loc[((train.metrostotales > 25) & (train.metrostotales <= 35)), 'metrostotales_agrupada'] = 30
train.loc[((train.metrostotales > 35) & (train.metrostotales <= 45)), 'metrostotales_agrupada'] = 40
train.loc[((train.metrostotales > 45) & (train.metrostotales <= 55)), 'metrostotales_agrupada'] = 50
train.loc[((train.metrostotales > 55) & (train.metrostotales <= 65)), 'metrostotales_agrupada'] = 60
train.loc[((train.metrostotales > 65) & (train.metrostotales <= 75)), 'metrostotales_agrupada'] = 70
train.loc[((train.metrostotales > 75) & (train.metrostotales <= 85)), 'metrostotales_agrupada'] = 80
train.loc[((train.metrostotales > 85) & (train.metrostotales <= 95)), 'metrostotales_agrupada'] = 90
train.loc[((train.metrostotales > 95) & (train.metrostotales <= 105)), 'metrostotales_agrupada'] = 100
train.loc[((train.metrostotales > 105) & (train.metrostotales <= 115)), 'metrostotales_agrupada'] = 110
train.loc[((train.metrostotales > 115) & (train.metrostotales <= 130)), 'metrostotales_agrupada'] = 125
train.loc[((train.metrostotales > 130) & (train.metrostotales <= 155)), 'metrostotales_agrupada'] = 150
train.loc[((train.metrostotales > 155) & (train.metrostotales <= 205)), 'metrostotales_agrupada'] = 200
train.loc[((train.metrostotales > 205) & (train.metrostotales <= 255)), 'metrostotales_agrupada'] = 250
train.loc[((train.metrostotales > 255) & (train.metrostotales <= 305)), 'metrostotales_agrupada'] = 300
train.loc[((train.metrostotales > 305) & (train.metrostotales <= 355)), 'metrostotales_agrupada'] = 350
train.loc[((train.metrostotales > 355) & (train.metrostotales <= 405)), 'metrostotales_agrupada'] = 400
train.loc[((train.metrostotales > 405) & (train.metrostotales <= 455)), 'metrostotales_agrupada'] = 450
train.loc[((train.metrostotales > 455) & (train.metrostotales <= 500)), 'metrostotales_agrupada'] = 500
train.loc[((train.metrostotales > 505) & (train.metrostotales <= 555)), 'metrostotales_agrupada'] = 550
train.loc[((train.metrostotales > 555) & (train.metrostotales <= 600)), 'metrostotales_agrupada'] = 600
train.loc[((train.metrostotales > 605) & (train.metrostotales <= 655)), 'metrostotales_agrupada'] = 650
train.loc[((train.metrostotales > 655) & (train.metrostotales <= 700)), 'metrostotales_agrupada'] = 700
train.loc[((train.metrostotales > 705) & (train.metrostotales <= 755)), 'metrostotales_agrupada'] = 750
train.loc[((train.metrostotales > 755) & (train.metrostotales <= 805)), 'metrostotales_agrupada'] = 800

In [10]:
train['fecha'] = pd.to_datetime(train['fecha'], infer_datetime_format=True)
train['anio'] = train['fecha'].dt.year
train['mes'] = train['fecha'].dt.month
train['dia'] = train['fecha'].dt.dayofweek

In [11]:
train.loc[train.tipodepropiedad.str.contains('COMERCIAL', na=False), 'tipodepropiedad'] = 'COMERCIO'
train.loc[train.tipodepropiedad.str.contains('CASA EN CONDOMINIO', na=False), 'tipodepropiedad'] = 'CONDOMINIO'

In [12]:
train_gruouped = train.groupby(['idzona','tipodepropiedad','habitaciones','anio'])['precio'].agg({'mean','median'}).reset_index()

In [13]:
train_gruouped.head()

,idzona,tipodepropiedad,habitaciones,anio,median,mean
0,22.0,APARTAMENTO,2.0,2013,1500000.0,1500000.0
1,22.0,APARTAMENTO,2.0,2014,961000.0,961000.0
2,22.0,APARTAMENTO,2.0,2015,2525000.0,2525000.0
3,22.0,APARTAMENTO,2.0,2016,2325000.0,1990427.5
4,22.0,APARTAMENTO,3.0,2013,2730000.0,2730000.0


In [14]:
train = pd.merge(train, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [15]:
test['lat'].fillna(0, inplace=True)
test['lng'].fillna(0, inplace=True)

test['titulo'] = test['titulo'].str.upper()
test['descripcion'] = test['descripcion'].str.upper()
test['direccion'] = test['direccion'].str.upper()
test['ciudad'] = test['ciudad'].str.upper()
test['provincia'] = test['provincia'].str.upper()
test['tipodepropiedad'] = test['tipodepropiedad'].str.upper()

test['provincia'] = test['provincia'].str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')

test['ciudad'] = test['ciudad'].str.normalize('NFKD')\
                                       .str.encode('ascii', errors='ignore')\
                                       .str.decode('utf-8')

In [16]:
test['excelente'] = 0
test.loc[test.descripcion.str.contains('VIGILANCIA', na=False), 'vigilancia'] = 1
test['vigilancia'] = 0
test.loc[test.descripcion.str.contains('EXCELENTE', na=False), 'excelente'] = 1
test['esquina'] = 0
test.loc[test.descripcion.str.contains('ESQUINA', na=False), 'esquina'] = 1
test['remodelada'] = 0
test.loc[test.descripcion.str.contains('REMODELADA', na=False), 'remodelada'] = 1
test['lujo'] = 0
test.loc[test.descripcion.str.contains('LUJO', na=False), 'lujo'] = 1

In [17]:
# AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
test['antiguedad_agrupada'] = 0

test.loc[test.antiguedad <= 5, 'antiguedad_agrupada'] = 0
test.loc[test.antiguedad > 95, 'antiguedad_agrupada'] = 100
test.loc[((test.antiguedad > 5) & (test.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
test.loc[((test.antiguedad > 15) & (test.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
test.loc[((test.antiguedad > 25) & (test.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
test.loc[((test.antiguedad > 35) & (test.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
test.loc[((test.antiguedad > 45) & (test.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
test.loc[((test.antiguedad > 55) & (test.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
test.loc[((test.antiguedad > 65) & (test.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
test.loc[((test.antiguedad > 75) & (test.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
test.loc[((test.antiguedad > 85) & (test.antiguedad <= 95)), 'antiguedad_agrupada'] = 90

In [18]:
# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
test['metroscubiertos_agrupada'] = 0

test.loc[test.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 0
test.loc[test.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 1000
test.loc[((test.metroscubiertos > 5) & (test.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 10
test.loc[((test.metroscubiertos > 15) & (test.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 20
test.loc[((test.metroscubiertos > 25) & (test.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 30
test.loc[((test.metroscubiertos > 35) & (test.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 40
test.loc[((test.metroscubiertos > 45) & (test.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 50
test.loc[((test.metroscubiertos > 55) & (test.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 60
test.loc[((test.metroscubiertos > 65) & (test.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 70
test.loc[((test.metroscubiertos > 75) & (test.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 80
test.loc[((test.metroscubiertos > 85) & (test.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 90
test.loc[((test.metroscubiertos > 95) & (test.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 100
test.loc[((test.metroscubiertos > 105) & (test.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 110
test.loc[((test.metroscubiertos > 115) & (test.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 125
test.loc[((test.metroscubiertos > 130) & (test.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 150
test.loc[((test.metroscubiertos > 155) & (test.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 200
test.loc[((test.metroscubiertos > 205) & (test.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 250
test.loc[((test.metroscubiertos > 255) & (test.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 300
test.loc[((test.metroscubiertos > 305) & (test.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 350
test.loc[((test.metroscubiertos > 355) & (test.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 400
test.loc[((test.metroscubiertos > 405) & (test.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 450
test.loc[((test.metroscubiertos > 455) & (test.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 500
test.loc[((test.metroscubiertos > 505) & (test.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 550
test.loc[((test.metroscubiertos > 555) & (test.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 600
test.loc[((test.metroscubiertos > 605) & (test.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 650
test.loc[((test.metroscubiertos > 655) & (test.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 700
test.loc[((test.metroscubiertos > 705) & (test.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 750
test.loc[((test.metroscubiertos > 755) & (test.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 800

In [19]:
# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
test['metrostotales_agrupada'] = 0

test.loc[test.metrostotales <= 5, 'metrostotales_agrupada'] = 0
test.loc[test.metrostotales > 805, 'metrostotales_agrupada'] = 1000
test.loc[((test.metrostotales > 5) & (test.metrostotales <= 15)), 'metrostotales_agrupada'] = 10
test.loc[((test.metrostotales > 15) & (test.metrostotales <= 25)), 'metrostotales_agrupada'] = 20
test.loc[((test.metrostotales > 25) & (test.metrostotales <= 35)), 'metrostotales_agrupada'] = 30
test.loc[((test.metrostotales > 35) & (test.metrostotales <= 45)), 'metrostotales_agrupada'] = 40
test.loc[((test.metrostotales > 45) & (test.metrostotales <= 55)), 'metrostotales_agrupada'] = 50
test.loc[((test.metrostotales > 55) & (test.metrostotales <= 65)), 'metrostotales_agrupada'] = 60
test.loc[((test.metrostotales > 65) & (test.metrostotales <= 75)), 'metrostotales_agrupada'] = 70
test.loc[((test.metrostotales > 75) & (test.metrostotales <= 85)), 'metrostotales_agrupada'] = 80
test.loc[((test.metrostotales > 85) & (test.metrostotales <= 95)), 'metrostotales_agrupada'] = 90
test.loc[((test.metrostotales > 95) & (test.metrostotales <= 105)), 'metrostotales_agrupada'] = 100
test.loc[((test.metrostotales > 105) & (test.metrostotales <= 115)), 'metrostotales_agrupada'] = 110
test.loc[((test.metrostotales > 115) & (test.metrostotales <= 130)), 'metrostotales_agrupada'] = 125
test.loc[((test.metrostotales > 130) & (test.metrostotales <= 155)), 'metrostotales_agrupada'] = 150
test.loc[((test.metrostotales > 155) & (test.metrostotales <= 205)), 'metrostotales_agrupada'] = 200
test.loc[((test.metrostotales > 205) & (test.metrostotales <= 255)), 'metrostotales_agrupada'] = 250
test.loc[((test.metrostotales > 255) & (test.metrostotales <= 305)), 'metrostotales_agrupada'] = 300
test.loc[((test.metrostotales > 305) & (test.metrostotales <= 355)), 'metrostotales_agrupada'] = 350
test.loc[((test.metrostotales > 355) & (test.metrostotales <= 405)), 'metrostotales_agrupada'] = 400
test.loc[((test.metrostotales > 405) & (test.metrostotales <= 455)), 'metrostotales_agrupada'] = 450
test.loc[((test.metrostotales > 455) & (test.metrostotales <= 500)), 'metrostotales_agrupada'] = 500
test.loc[((test.metrostotales > 505) & (test.metrostotales <= 555)), 'metrostotales_agrupada'] = 550
test.loc[((test.metrostotales > 555) & (test.metrostotales <= 600)), 'metrostotales_agrupada'] = 600
test.loc[((test.metrostotales > 605) & (test.metrostotales <= 655)), 'metrostotales_agrupada'] = 650
test.loc[((test.metrostotales > 655) & (test.metrostotales <= 700)), 'metrostotales_agrupada'] = 700
test.loc[((test.metrostotales > 705) & (test.metrostotales <= 755)), 'metrostotales_agrupada'] = 750
test.loc[((test.metrostotales > 755) & (test.metrostotales <= 805)), 'metrostotales_agrupada'] = 800

In [20]:
test['fecha'] = pd.to_datetime(test['fecha'], infer_datetime_format=True)
test['mes'] = test['fecha'].dt.month
test['dia'] = test['fecha'].dt.dayofweek
test['anio'] = test['fecha'].dt.year

In [21]:
test.loc[test.tipodepropiedad.str.contains('COMERCIAL', na=False), 'tipodepropiedad'] = 'COMERCIO'
test.loc[test.tipodepropiedad.str.contains('CASA EN CONDOMINIO', na=False), 'tipodepropiedad'] = 'CONDOMINIO'

In [22]:
test = pd.merge(test, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])

In [23]:
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,remodelada,lujo,antiguedad_agrupada,metroscubiertos_agrupada,metrostotales_agrupada,mes,dia,anio,median,mean
0,4941,"CASA EN VENTA EN MIGUEL HIDALGO, DISTRITO FEDERAL",<P>EXCELENTE CASA ESTILO MODERNO.</P>,CASA,BOSQUE DE CEDROS,MIGUEL HIDALGO,DISTRITO FEDERAL,29.0,3.0,NaN,...,0,0,30,300,0,7,5,2013,NaN,NaN
1,51775,DEPARTAMENTOS EN VENTA EN MONTEBELLO,<P>DEPARTAMENTO UNA RECAMARA:\n</P><P>DEPARTAM...,APARTAMENTO,NaN,MARIDA,YUCATAN,NaN,1.0,1.0,...,0,1,0,70,70,10,5,2015,959000.0,9.556667e+05
2,115253,DEPARTAMENTO NUEVO DELEGACIóN COYOACáN DE 87 M...,"DEPARTAMENTO NUEVO DE 87.06 M2, 1 CAJóN DE EST...",APARTAMENTO,"PUEBLO DE LOS REYES, COYOACáN, MEXICO D.F.",COYOACAN,DISTRITO FEDERAL,0.0,2.0,1.0,...,0,0,0,90,100,5,5,2015,1745000.0,1.811200e+06
3,299321,DEPARTAMENTO EN VENTA EN ACAPULCO,<P> RAíCES DV001 PRECIOSO DEPARTAMENTO TIPO K...,APARTAMENTO,NaN,ACAPULCO DE JUAREZ,GUERRERO,2.0,2.0,2.0,...,0,0,0,90,90,4,3,2015,1337500.0,1.337500e+06
4,173570,BONITA CASA SOLA EQUIPADA DE DOS NIVELES EN LO...,"<P>CASA SOLA, BONITA DE DOS REC&AACUTE;MARAS U...",CASA,CEDROS,TULTITLAN,EDO. DE MAXICO,10.0,2.0,1.0,...,0,0,10,80,80,8,3,2013,552500.0,5.525000e+05


In [24]:
## LIMPIAMOS LOS VALORES NULOS O NAN QUE NO NOS SIRVEN. (CRITERIO)
train['mean'].fillna((train['mean'].mean()), inplace=True)
train['median'].fillna((train['median'].mean()), inplace=True)
train['antiguedad'].fillna((train['antiguedad'].mean()), inplace=True)
train['habitaciones'].fillna((train['habitaciones'].mean()), inplace=True)
train['garages'].fillna((train['garages'].mean()), inplace=True)
train['banos'].fillna((train['banos'].mean()), inplace=True)
train['metroscubiertos'].fillna((train['metroscubiertos'].mean()), inplace=True)
train['metrostotales'].fillna((train['metrostotales'].mean()), inplace=True)
                       
test['mean'].fillna((test['mean'].mean()), inplace=True)
test['median'].fillna((test['median'].mean()), inplace=True)
test['antiguedad'].fillna((test['antiguedad'].mean()), inplace=True)
test['habitaciones'].fillna((test['habitaciones'].mean()), inplace=True)
test['garages'].fillna((test['garages'].mean()), inplace=True)
test['banos'].fillna((test['banos'].mean()), inplace=True)
test['metroscubiertos'].fillna((test['metroscubiertos'].mean()), inplace=True)
test['metrostotales'].fillna((test['metrostotales'].mean()), inplace=True)


train.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
test.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,remodelada,lujo,antiguedad_agrupada,metroscubiertos_agrupada,metrostotales_agrupada,mes,dia,anio,median,mean
0,4941,"CASA EN VENTA EN MIGUEL HIDALGO, DISTRITO FEDERAL",<P>EXCELENTE CASA ESTILO MODERNO.</P>,CASA,BOSQUE DE CEDROS,MIGUEL HIDALGO,DISTRITO FEDERAL,29.000000,3.000000,1.547605,...,0,0,30,300,0,7,5,2013,2.719107e+06,2.787450e+06
1,51775,DEPARTAMENTOS EN VENTA EN MONTEBELLO,<P>DEPARTAMENTO UNA RECAMARA:\n</P><P>DEPARTAM...,APARTAMENTO,1,MARIDA,YUCATAN,8.152599,1.000000,1.000000,...,0,1,0,70,70,10,5,2015,9.590000e+05,9.556667e+05
2,115253,DEPARTAMENTO NUEVO DELEGACIóN COYOACáN DE 87 M...,"DEPARTAMENTO NUEVO DE 87.06 M2, 1 CAJóN DE EST...",APARTAMENTO,"PUEBLO DE LOS REYES, COYOACáN, MEXICO D.F.",COYOACAN,DISTRITO FEDERAL,0.000000,2.000000,1.000000,...,0,0,0,90,100,5,5,2015,1.745000e+06,1.811200e+06
3,299321,DEPARTAMENTO EN VENTA EN ACAPULCO,<P> RAíCES DV001 PRECIOSO DEPARTAMENTO TIPO K...,APARTAMENTO,ALUMNOS,ACAPULCO DE JUAREZ,GUERRERO,2.000000,2.000000,2.000000,...,0,0,0,90,90,4,3,2015,1.337500e+06,1.337500e+06
4,173570,BONITA CASA SOLA EQUIPADA DE DOS NIVELES EN LO...,"<P>CASA SOLA, BONITA DE DOS REC&AACUTE;MARAS U...",CASA,CEDROS,TULTITLAN,EDO. DE MAXICO,10.000000,2.000000,1.000000,...,0,0,10,80,80,8,3,2013,5.525000e+05,5.525000e+05
5,30862,CASA EN VENTA PARQUES DE SANTA MARIA 3REC. POR...,A UN COSTADO DE PARQUES DE SANTA MARíA \nCON ...,CASA,FRESNO 2601,ZAPOPAN,JALISCO,10.000000,3.000000,1.000000,...,0,0,10,200,150,1,0,2016,2.719107e+06,2.787450e+06
6,244471,CASA EN VENTA EN LA COL. CORTIJO DEL RIO,"<P>EXCELENTE PROPIEDAD EN ZONA SUR, COCINA AMP...",CASA,SENDA DE LA CREACIÓN,MONTERREY,NUEVO LEA3N,20.000000,3.000000,1.547605,...,0,0,20,250,200,6,4,2016,2.600000e+06,2.722675e+06
7,127794,"CASAS JAVER, CASAS DE 3 RECAMARAS. A TAN SOLO ...",BOSQUES DE LERMA ES UN EXCLUSIVO FRACCIONAMIEN...,CASA,LERMA ESTADO DE MEXICO,LERMA,EDO. DE MAXICO,0.000000,2.000000,1.000000,...,0,0,0,60,60,1,5,2016,7.935000e+05,7.935000e+05
8,71558,HERMOSA CASA CON ALBERCA Y ACCESO A LA PLAYA,"HERMOSA RESIDENCIA, CON ALBERCA Y ACCESO A LA ...",CASA,AMERICAS 2,VERACRUZ,VERACRUZ,2.000000,3.000000,2.000000,...,0,0,0,300,200,7,3,2016,1.595000e+06,1.770333e+06
9,218011,CASA EN SM 30 CANCUN,DESCRIPCIóN: \nPLANTA BAJA \n2 RECAMARAS \n1 B...,CASA,YALAHAN,CANCAoN,QUINTANA ROO,20.000000,3.000000,1.000000,...,0,0,20,125,200,10,0,2015,2.719107e+06,2.787450e+06


In [25]:
## COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)

train['provincia'] = train['provincia'].astype('category')
dummies = pd.get_dummies(train['provincia'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

train['tipodepropiedad'] = train['tipodepropiedad'].astype('category')
dummies = pd.get_dummies(train['tipodepropiedad'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

train['anio'] = train['anio'].astype('category')
dummies = pd.get_dummies(train['anio'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

#train['ciudad'] = train['ciudad'].astype('category')
#dummies = pd.get_dummies(train['ciudad'], drop_first=False)
#train = pd.concat([train, dummies], axis=1)
#dummies = ''

In [26]:
## COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)

test['provincia'] = test['provincia'].astype('category')
dummies = pd.get_dummies(test['provincia'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

test['tipodepropiedad'] = test['tipodepropiedad'].astype('category')
dummies = pd.get_dummies(test['tipodepropiedad'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

test['anio'] = test['anio'].astype('category')
dummies = pd.get_dummies(test['anio'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

#test['ciudad'] = test['ciudad'].astype('category')
#dummies = pd.get_dummies(test['ciudad'], drop_first=False)
#test = pd.concat([test, dummies], axis=1)
#dummies = ''

### =====================================================================
### LIMPIEZA Y COLUMNAS RESULTANTES.
### =====================================================================

In [27]:
## ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS.
## PERO TENERLAS PRESENTES PARA HACER UN TRABAJO POSTERIOR.

train.drop('titulo', axis=1, inplace=True)
train.drop('descripcion', axis=1, inplace=True)
train.drop('tipodepropiedad', axis=1, inplace=True)
train.drop('direccion', axis=1, inplace=True)
train.drop('ciudad', axis=1, inplace=True)
train.drop('provincia', axis=1, inplace=True)
train.drop('fecha', axis=1, inplace=True)

test.drop('titulo', axis=1, inplace=True)
test.drop('descripcion', axis=1, inplace=True)
test.drop('tipodepropiedad', axis=1, inplace=True)
test.drop('direccion', axis=1, inplace=True)
test.drop('ciudad', axis=1, inplace=True)
test.drop('provincia', axis=1, inplace=True)
test.drop('fecha', axis=1, inplace=True)

In [28]:
## ELIMINAMOS COLUMNAS NUMÉRICAS QUE CARECEN DE SENTIDO.

train.drop('idzona', axis=1, inplace=True)
train.drop('lat', axis=1, inplace=True)
train.drop('lng', axis=1, inplace=True)

test.drop('idzona', axis=1, inplace=True)
test.drop('lat', axis=1, inplace=True)
test.drop('lng', axis=1, inplace=True)

train.drop('dia', axis=1, inplace=True)
train.drop('mes', axis=1, inplace=True)
train.drop('anio', axis=1, inplace=True)

test.drop('dia', axis=1, inplace=True)
test.drop('mes', axis=1, inplace=True)
test.drop('anio', axis=1, inplace=True)

In [29]:
test['HOSPEDAJE'] = 0
test['GARAGE'] = 0

In [30]:
train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 80 columns):
id                               240000 non-null int64
antiguedad                       240000 non-null float64
habitaciones                     240000 non-null float64
garages                          240000 non-null float64
banos                            240000 non-null float64
metroscubiertos                  240000 non-null float64
metrostotales                    240000 non-null float64
gimnasio                         240000 non-null float64
usosmultiples                    240000 non-null float64
piscina                          240000 non-null float64
escuelascercanas                 240000 non-null float64
centroscomercialescercanos       240000 non-null float64
precio                           240000 non-null float64
excelente                        240000 non-null int64
vigilancia                       240000 non-null int64
esquina                          240000 

In [31]:
test.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 79 columns):
id                               60000 non-null int64
antiguedad                       60000 non-null float64
habitaciones                     60000 non-null float64
garages                          60000 non-null float64
banos                            60000 non-null float64
metroscubiertos                  60000 non-null float64
metrostotales                    60000 non-null float64
gimnasio                         60000 non-null float64
usosmultiples                    60000 non-null float64
piscina                          60000 non-null float64
escuelascercanas                 60000 non-null float64
centroscomercialescercanos       60000 non-null float64
excelente                        60000 non-null int64
vigilancia                       60000 non-null int64
esquina                          60000 non-null int64
remodelada                       60000 non-null int64
lujo 

In [32]:
train.to_csv('DATA/train.csv', index=False)
test.to_csv('DATA/test.csv', index=False)